In [1]:
!python3 -m spacy download en_core_web_sm



     |████████████████████████████████| 13.6 MB 4.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


ModuleNotFoundError: No module named 'en_core_wen_sm'

In [7]:
import spacy 
import en_core_web_sm 
nlp = en_core_web_sm.load()

In [3]:
import pickle
# Load: insert variable to load pickle into
train = pickle.load(open('../extra/pickle/train_bal.pickle', 'rb'))
valid = pickle.load(open('../extra/pickle/val_bal.pickle', 'rb'))

In [4]:
train.head()

,tweet,target
0,@ICC you should be worried about other things ...,1
1,@amberhasalamb Can you comment on Ford doing t...,1
2,@MalcolmNance @montanaisthebes I agree with ma...,1
3,And now for the dish that best represents the ...,1
4,.@mark_kaiserr Im not sexist but every food pl...,1


In [6]:
train.tweet[0]

'@ICC you should be worried about other things rather than a sign of patriotism on @msdhoni gloves huh #DhoniKeepsTheGlove #INDvsAUS #ICCWorldCup2019 https://t.co/dPlEcrWHYt'

In [17]:
import pandas as pd
s = train.tweet[0]
doc = nlp(s)

rows = []
for token in doc:
    rows.append([token, token.lemma_, token.is_stop])

df = pd.DataFrame(rows, columns=['token', 'lemma', 'stopwords'])
df

,token,lemma,stopwords
0,@ICC,@ICC,False
1,you,you,True
2,should,should,True
3,be,be,True
4,worried,worried,False
5,about,about,True
6,other,other,True
7,things,thing,False
8,rather,rather,True
9,than,than,True


In [20]:
import re

re_token_match = spacy.tokenizer._get_regex_pattern(nlp.Defaults.token_match)
re_token_match = f"({re_token_match}|#\\w+)"
nlp.tokenizer.token_match = re.compile(re_token_match).match

In [21]:
doc = nlp(s)

rows = []
for token in doc:
    rows.append([token, token.lemma_, token.is_stop])

df = pd.DataFrame(rows, columns=['token', 'lemma', 'stopwords'])
df

,token,lemma,stopwords
0,@ICC,@ICC,False
1,you,you,True
2,should,should,True
3,be,be,True
4,worried,worried,False
5,about,about,True
6,other,other,True
7,things,thing,False
8,rather,rather,True
9,than,than,True


In [22]:
def preprocess(s, nlp, features):
    s = s.lower()
    doc = nlp(s)
    lemmas = []
    for token in doc:
        lemmas.sppend(token.lemma_)
    features |= set(lemmas)
    freq = {"#":0,"@":0,"URL":0}
    for words in lemmas:
        freq[str(words)] = 0
    for token in doc:
        if '#' in str(token): freq['#'] += 1
        if "@" in str(token): freq['@'] += 1
        if "https://" in str(token): freq['URL'] += 1
        freq[str(token.lemma_)] += 1

    return features, freq